In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_pickle('../../data/ava_st1_ns4_56.pkl')
df.head


<bound method NDFrame.head of        apcp_sf1_1  apcp_sf2_1 apcp_sf3_1  apcp_sf4_1  apcp_sf5_1  dlwrf_s1_1  \
V1       0.000000         0.0          1    0.000000    0.000000  256.492673   
V2       0.000000         0.0          1    0.017273    0.139091  257.998596   
V3       0.000000         0.0          1    0.000000    0.000000  219.280002   
V4       0.004545         0.0          1    0.000000    0.000000  267.863045   
V5       0.000000         0.0          1    0.000000    0.000000  238.162747   
...           ...         ...        ...         ...         ...         ...   
V4376    0.010909         0.0          1    0.000000    0.000000  278.168651   
V4377    0.000000         0.0          1    0.000000    0.000000  251.551092   
V4378    0.000000         0.0          1    0.000000    0.000000  269.446164   
V4379    0.000000         0.0          1    0.000000    0.000000  268.862049   
V4380    0.000000         0.0          1    0.000000    0.000000  269.112621   

       dl

: 

: 

: 

In [ ]:
x = df.drop('energy', axis=1)
y = df['energy']


: 

: 

: 

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split


: 

: 

: 

In [ ]:
preprocessor = ColumnTransformer(
    transformers = [
        ('STscaler', StandardScaler(), make_column_selector(dtype_include=np.float64)),
        ('MMscaler', MinMaxScaler(), make_column_selector(dtype_include=np.int64))
    ],
    remainder = 'passthrough'
)

preprocessor.fit(x)

X =  preprocessor.transform(x)



: 

: 

: 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


: 

: 

: 

In [ ]:
from sklearn.tree import DecisionTreeRegressor

: 

: 

: 

In [ ]:
tree = DecisionTreeRegressor(max_depth=5)
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)

: 

: 

: 

: 

: 

: 

: 

: 

Creating a Dummy Classifier to see if the model is better than random chance.

In [ ]:
from sklearn.dummy import DummyClassifier

: 

: 

: 

: 

: 

: 

: 

: 

In [ ]:
dummy_classifier = DummyClassifier(strategy='most_frequent')
dummy_classifier.fit(X_train, y_train)
y_pred_dummy = dummy_classifier.predict(X_test)

: 

: 

: 

: 

: 

: 

: 

: 

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error


: 

: 

: 

: 

: 

: 

: 

: 

In [ ]:
mse = mean_squared_error(y_test, y_pred)
mse_dummy = mean_squared_error(y_test, y_pred_dummy)
print("The model's mse: ", mse)
print("The dummy's mse: ", mse_dummy)
print("Relative error: ", mse/mse_dummy)




The model's mse:  13821587392139.982
The dummy's mse:  94919852786584.11
Relative error:  0.14561324092248817


: 

: 

: 

: 

: 

: 

: 

: 

In [ ]:
r2 = r2_score(y_test, y_pred)
r2_dummy = r2_score(y_test, y_pred_dummy)
print("The model's r2: ", r2)   
print("The dummy's r2: ", r2_dummy)


The model's r2:  0.7788114803425561
The dummy's r2:  -0.5190137810007638


: 

: 

: 

: 

: 

: 

: 

: 

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
mae_dummy = mean_absolute_error(y_test, y_pred_dummy)
print("The model's mae: ", mae)
print("The dummy's mae: ", mae_dummy)
print("Relative error: ", mae/mae_dummy)

The model's mae:  2575515.3515981734
The dummy's mae:  8069114.164383561
Relative error:  0.3191819199889743


: 

: 

: 

: 

: 

: 

: 

: 